the goal of this notebook is to be able to:

- take a timeseries that contains gaps
- train a GPFA Learner using the available data
- impute the gap, by using the predictions of the learner
- to improve the imputation by conditioning the distribution on the other observation at the same time stamp

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#| hide
#| default_exp imputation

In [3]:
#| export
from gpfa_imputation.learner import *
from gpfa_imputation.data_preparation import *

import torch

import pandas as pd
import numpy as np
import sklearn

from fastcore.foundation import patch

import matplotlib.pyplot as plt
import altair as alt
from altair import datum

from functools import lru_cache

In [4]:
torch.manual_seed(277); # set seed for reproducibility of examples

## GPFA Imputation

This is the core class that does the imputation using a GPFA

The inputs is:

- a dataframe containing the observed data, where the row with missing data have been removed
- a vector of times where the data is missing

it returns:

- a complete dataframe with the prediction of the model

The goal is that `GPFAImputation` takes as imput a dataframe containing missing values and then it imputes them using GPFALearner.
Therefore it needs to divide the dataframe in 3 sections:

- training data (rows with no NAs)
- times to be imputed (rows with some NAs)
- observations (variables in the pred rows that are not missing) for conditional predictions

In [5]:
t_df = pd.DataFrame([
    [1., 3., 4.],
    [2., 6., np.nan],
    [np.nan, np.nan, np.nan],
    [np.nan, 8., np.nan],
    [3., 4., 5.]
]
)

In [6]:
train_idx = ~t_df.isna().any(1)

/tmp/ipykernel_27606/3664883938.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.any and Series.any will be keyword-only.
  train_idx = ~t_df.isna().any(1)


In [7]:
t_df[train_idx]

,0,1,2
0,1.0,3.0,4.0
4,3.0,4.0,5.0


In [8]:
pred_data = t_df[~train_idx].to_numpy() 

In [9]:
pred_data

array([[ 2.,  6., nan],
       [nan, nan, nan],
       [nan,  8., nan]])

In [10]:
idx_cond = ~t_df[~train_idx].isna()

In [11]:
idx_cond

,0,1,2
1,True,True,False
2,False,False,False
3,False,True,False


In [12]:
idx_cond = idx_cond.to_numpy().flatten() # learner needs a 1D index

pred_data.flatten()[idx_cond]

train_idx = t_df.isna().any(axis=1)

torch.tensor(~t_df[~train_idx].isna().to_numpy().flatten())

tensor([True, True, True, True, True, True])

Implement this into a function

In [33]:
#| exports
class GPFAImputation:
    def __init__(
        self,
        data: pd.DataFrame , #observed data with missing data as NA
        complete_data: pd.DataFrame = None, # Optional complete dataframe (for testing)
        cuda = True # Use GPU?
    ):
        self.data = data
        self.data_complete = complete_data # TODO polish this, is this tidy or wide? check has required cols
        
        device = 'cuda' if cuda else 'cpu'
        
        self.T = torch.arange(0, len(data), dtype=torch.float32, device=device) # time is encoded with a increase of 1
        
        # Training data
        self.train_idx = ~self.data.isna().any(axis=1)
        self.train_data = torch.tensor(self.data[self.train_idx].to_numpy().astype(np.float32), device=device)
        self.train_T = self.T[self.train_idx]
        
        self.learner = GPFALearner(X = self.train_data, T = self.train_T)
        

        # Prediction data
        self.pred_T = self.T[~self.train_idx]
        self.cond_idx = torch.tensor(~self.data[~self.train_idx].isna().to_numpy().flatten(), device=device) # conditional obsevations
        self.cond_obs = torch.tensor(self.data[~self.train_idx].to_numpy().astype(np.float32).flatten()[self.cond_idx.cpu()], device=device)
        
        if cuda: self.learner.cuda()
        
        
    @lru_cache
    def impute(self,
               add_time = True, # add column with time?
               tidy = True # tidy data?
               ):
        
        if not hasattr(self, "pred"):
            self.learner.train()
            self.pred = self.learner.predict(self.pred_T, obs = self.cond_obs, idx = self.cond_idx)
        
        if tidy: return self._impute_tidy(add_time)
        else: return self._impute_wide(add_time)
        
        
    def _impute_wide(self, add_time):
        """ Impute in wide format"""
        
        imp_data = self.data.copy()
        for col_idx, col_name in enumerate(imp_data.columns):
            imp_data.loc[~self.train_idx, col_name] = self.pred.mean[:, col_idx].cpu().numpy()
            imp_data.loc[~self.train_idx, col_name + "_std"] = self.pred.std[:, col_idx].cpu().numpy()
        
        idx_vars = []
        if add_time:
            imp_data["time"] = self.T.cpu()
            idx_vars.append("time")
        
        return imp_data 
    
    def _impute_tidy(self, add_time):
        """ transform the pred output into a tidy dataframe suitable for plotting"""
        feature_names = self.data.columns

        pred_mean = pd.DataFrame(self.pred.mean.cpu(), columns = feature_names).assign(time = self.pred_T.cpu()).melt("time", value_name="mean")
        pred_std = pd.DataFrame(self.pred.std.cpu(), columns = feature_names).assign(time = self.pred_T.cpu()).melt("time", value_name="std")
        
        pred = pd.merge(pred_mean, pred_std, on=['time', 'variable'])  
        
        train_data = self.data[self.train_idx].assign(time = self.train_T.cpu()).melt("time", value_name = "mean")
               
        imp_data = pd.concat((train_data, pred))
        
        self.pred_wide = imp_data
        
        return imp_data
    

In [34]:
fd = GPFADataTest.generate(2, 10, Lambda=[1,2.]).add_random_missing()

In [35]:
~fd.data.isna().any(axis=1)

0     True
1     True
2     True
3     True
4    False
5    False
6     True
7    False
8     True
9     True
dtype: bool

In [36]:
fd.data

,x0,x1
0,-0.302333,-0.055494
1,0.157988,0.243542
2,-0.115275,-0.000321
3,0.603859,1.032325
4,NaN,NaN
5,0.919418,NaN
6,-0.918922,-1.617750
7,NaN,NaN
8,-1.010151,-1.763201
9,0.960713,1.850948


In [37]:
imp = GPFAImputation(fd.data, fd.tidy_df(complete=True, is_missing=True))

In [38]:
imp

In [39]:
next(imp.learner.model.parameters()).device

device(type='cuda', index=0)

In [40]:
imp.learner.cuda()

In [41]:
pred = imp.impute()
pred

  0%|          | 0/100 [00:00<?, ?it/s]

,time,variable,mean,std
0,0.0,x0,-0.302333,NaN
1,1.0,x0,0.157988,NaN
2,2.0,x0,-0.115275,NaN
3,3.0,x0,0.603859,NaN
4,6.0,x0,-0.918922,NaN
5,8.0,x0,-1.010151,NaN
6,9.0,x0,0.960713,NaN
7,0.0,x1,-0.055494,NaN
8,1.0,x1,0.243542,NaN
9,2.0,x1,-0.000321,NaN


In [ ]:
imp.impute(tidy=False)

### Prediction plot

In [ ]:
#| exports
def _plot_variable(imp, complete, variable, y_label="", sel=None, properties = {}):
    
    imp = imp[imp.variable == variable]

    
    error = alt.Chart(imp).mark_errorband().encode(
        x = "time",    
        y = alt.Y("err_low:Q", title = y_label, scale=alt.Scale(zero=False)),
        y2 = "err_high:Q",
        color=alt.Color("variable",
                        legend = alt.Legend(title=["Line: pred. mean", "area: +/- 2 std", "(variable)"])
                       ),
        tooltip = alt.Tooltip(['std', 'mean'], format=".4")
    ).transform_calculate(
        err_low = "datum.mean - 2 * datum.std",
        err_high = "datum.mean + 2 * datum.std"
    ).properties( **properties)

    pred = alt.Chart(imp).mark_line().encode(
        x = "time",    
        y = alt.Y("mean:Q", title = y_label, scale=alt.Scale(zero=False)),
        color="variable",
    ).add_selection(
        sel if sel is not None else alt.selection_interval(bind="scales")
    ).properties(title = variable)

    base_plot = error + pred
    
    if complete is not None:

        complete = complete[complete.variable == variable]
        truth_plt = alt.Chart(complete).mark_point(
            color='black',
            strokeWidth = 1,
            fillOpacity = 1
        ).encode(
            x = "time",
            y = alt.Y("value", title = y_label, scale=alt.Scale(zero=False)),
            fill= alt.Fill("is_missing", scale = alt.Scale(range=["#ffffff00", "black"]),
                           legend = alt.Legend(title =["Observed data","(is missing)"])),
            shape = "is_missing",
        )

        base_plot = truth_plt + base_plot
        
    return base_plot
    

In [ ]:
imp_d = imp.impute(tidy=True, add_time=True)

_plot_variable(imp_d, None, "x1")

In [ ]:
#| exports
@patch()
def plot_pred(
    self: GPFAImputation,
    complete = True, # Optional true data to be plotted agaist predictions
    units: dict = None, # Optional dict where keys are col name and value the unit (y axis labels)
    n_cols: int = 2,
    bind_interaction: bool =True, # Whether the sub-plots for each variable should be connected for zooming/panning
    properties:dict = {} # addtional properties (eg. size) for altair plot
):
    "Plot the prediction for each variable, optionally including true values"
    imp = self._impute_tidy(add_time=True) if hasattr(self, "pred") else self.impute(tidy=True, add_time=True)
    
   
    plot_list = [alt.hconcat() for _ in range(0, imp.shape[0], n_cols)]
    selection_scale = alt.selection_interval(bind="scales", encodings=['x']) if bind_interaction else None
    for idx, variable in enumerate(pd.unique(imp.variable)):
        plot_list[idx // n_cols] |= _plot_variable(imp,
                                                   self.data_complete if complete else None,
                                                   variable,
                                                   y_label = f"{variable} [{units[variable]}]" if units is not None else variable,
                                                   sel = selection_scale, properties=properties)
    
    plot = alt.vconcat(*plot_list)
    
    return plot

In [ ]:
imp.plot_pred(units = {'x0': 'x0 units', 'x1': 'x1 unitssss'})

The code is running correctly and as expected around the missing data point the error is band is wider

### Repr

add `__repr__` and `__str__` to imputation objects

In [ ]:
imp.data

In [ ]:
#| exports
@patch
def __repr__(self: GPFAImputation):
    return f"""GPFA Imputation:
    N obs: {self.data.shape[0]}
    N features {self.data.shape[1]} ({', '.join(self.data.columns)})
    N missing observations {(~self.cond_idx).sum()}"""

@patch
def __str__(self: GPFAImputation):
    return self.__repr__()

In [ ]:
imp

In [ ]:
str(imp)

### Metrics

In [ ]:
#| exports
@patch
def compute_metric(self: GPFAImputation,
                   metric, # function that takes as argument true and pred and returns the metric
                   metric_name = 'metric'):
    pred = self.impute(tidy=True, add_time=True)
    
    df = pd.merge(pred, self.data_complete, on = ['time','variable'])
    
    vars = []
    
    for var in df.variable.unique():
        df_var = df[df.variable == var]
        vars.append({'variable': var,
                      metric_name: metric(df_var['value'], df_var['mean'])})
    
    return pd.DataFrame(vars)

In [ ]:
#| exports
@patch
def rmse(self: GPFAImputation):
    
    return self.compute_metric(lambda x, y: np.sqrt(sklearn.metrics.mean_squared_error(x,y)), "rmse")
    

In [ ]:
imp.rmse()

In [ ]:
#| exports
@patch
def r2(self: GPFAImputation):
    return self.compute_metric(sklearn.metrics.r2_score, "r2")

In [ ]:
imp.r2()

## CPU 

check that the CPU support is still working

In [ ]:
imp_cpu = GPFAImputation(fd.data, fd.tidy_df(complete=True, is_missing=True), cuda=False)

In [ ]:
imp_cpu.impute()

In [1]:
#| hide
from nbdev import nbdev_export
nbdev_export()